# Advektionsgleichung (1D)

Bisher haben wir die Energieerhaltungsgleichung nur ohne den Transport von Materialien (der Advektion)
betrachtet. Häufig haben wir aber ein Problem in dem sich das Material bewegt und bestimmte Größen, wie
z.B. die Temperatur, Dichte, etc., transportiert werden müssen (z.B. in Mantleplumes). Generell gesehen, ist die
Mantelkonvektion ein Beispiel eines Systems, in welchem die Temperatur sowohl durch **Diffusion** (vor allem in
den Grenzschichten) als auch **Advektion** (vor allem in Inneren) transportiert wird.

Betrachten wir im Folgenden nur die **Advektion** alleine (d.h. k, κ = 0).

Im 1-D Fall reduziert sich die Gleichung dadurch zur reinen Advektionsgleichung:

$$
\frac{\partial{T}}{\partial{t}}=-v_x\frac{\partial{T}}{\partial{x}}. \tag{1}
$$

Diese Gleichung lässt sich durch verschiedene Diskretisierungsverfahren numerisch lösen und wir wollen
verschiedene Verfahren programieren und auf ein bestimmtes Advektionsproblem anwenden.

Betrachten wir dazu zwei 1-D Probleme (z.B. ein horizontales Temperaturprofil) mit bestimmten Anomalien:

&emsp;    • Einen Gaußchen Temperaturverlauf (ein glatter Übergang):<br>
<img src="../Figures/Exercise06_gaussian.png" alt="drawing" width="400"/> <br>


&emsp;    • Eine block-förmige Temperaturanomalie (ein sehr scharfer Übergang):<br>
<img src="../Figures/Exercise06_block.png" alt="drawing" width="400"/> <br>


Laden wir erst einmal die notwendigen Module: 

In [157]:
using Plots #, Dierckx
using GeoModBox.AdvectionEquation.OneD

Definieren wir nun unsere Geometrie und die benötigten numerischen Parameter (d.h.
Gitterauflösung, Gitter, etc.):

In [158]:
# Geometrische Konstanten ----------------------------------------------- #
xmin    =   0                           #   [ m ]
xmax    =   40                          #   [ m ]
# ----------------------------------------------------------------------- #
# Numerische Konstanten ------------------------------------------------- #
nc      =   100                         #   Anzahl der Gitterpunkte
Δx      =   xmax/nc                     #   Gitterlänge
ind     =   1:nc   
# ---
xc      =   xmin+Δx/2:Δx:xmax-Δx/2      #   x-Koordinate
xcwe    =   xmin-Δx/2:Δx:xmax+Δx/2      #   x-Koordinate
X       =   zeros(nc)
# ----------------------------------------------------------------------- #
# Maximale Laufzeit des Models ------------------------------------------ #
tmax    =   40.0                        #   [ s ]
# ----------------------------------------------------------------------- #
# Horizontale Geschwindigkeit ------------------------------------------- #
vx      =   1.0                         #   [ m/s ]
# ----------------------------------------------------------------------- #
# Definition der Zeitschrittlänge --------------------------------------- #
Δtfac   =   0.8                         #   Courant-Kriterium
Δt      =   Δtfac*Δx/abs(vx)
nt      =   ceil(Int,tmax/Δt)           #   Anzahl der Zeitschritte
# ----------------------------------------------------------------------- #

125

Nun können wir zum einen die FD-Methode wählen ('FTCS', 'upwind', 'downwind', 'lax', 'slf', 'semilag' - die
Gleichungen und Erläuterungen dazu sind im Detail in den Folien der Vorlesung zu finden) und zum anderen
das Anfangsprofil wählen ('block' oder 'gaussian'):

In [159]:
FD          =   (Method     = (Adv=:semilag,),)
Ini         =   (T=:gaussian,)

(T = :gaussian,)

Zur Visualisierung und zum speichern der Animationsdatei muss der Speicherort festgelegt werden:

In [160]:
# Animationssettings ---------------------------------------------------- #
path        =   string("./Results/")
anim        =   Plots.Animation(path, String[] )
filename    =   string("06_1D_advection_",Ini.T,"_",FD.Method.Adv)
save_fig    =   1
# ----------------------------------------------------------------------- #

1

Falls wir die Tracer Methode verwenden wollen, müssen wir noch die Anzahl der Tracer pro Gitterlänge
festlegen:

In [161]:
# Tracer advection method ----------------------------------------------- #
nmx         =   3       #   Number of tracers per "cell"
# ----------------------------------------------------------------------- #

3

Wollen wir nun die Anfangsbedingung (d.h. das Anfangstemperaturprofil) definieren und plotten:

In [162]:
if Ini.T==:block 
    # Hintergrundtemperatur ----------------------------------------- #
    Tb      =   1000                #   [ K ]
    
    # Lokalität und Intensität der Temperaturanomalie --------------- #
    xTl     =   (xmax-xmin)/10
    xTr     =   xTl + (xmax-xmin)/10
    Ta      =   1500                #   [ K ]
    
    # Erstellung des Anfangstemperaturprofiles ---------------------- #
    T       =   Tb.*ones(nc)        
    T[xc.>=xTl .&& xc .<= xTr ]       .=  Ta
    Tmin    =   minimum(T)
    Tmax    =   maximum(T)
    tc      =   100
elseif Ini.T==:gaussian
    # Gaußsche Temperature Verteilung ------------------------------- #
    Tb      =   1000                #   Hintergrundtempertur
    Ampl    =   500                 #   Amplitude
    sigma   =   1                   #   Standard Abweichung
    xcG     =   (xmax-xmin)/10      #   x-Koordinate des Maximums
    T       =   zeros(nc)
    @. T    =  Tb + Ampl*exp(-((xc - xcG)^2)/sigma^2)
    
    Tmin    =   minimum(T)
    Tmax    =   maximum(T)
    tc      =   Ampl
end
TWE             =   zeros(nc+2)
TWE[2:end-1]    .=  T
TWE[1]          =   T[end]
TWE[end]        =   T[1]
TWE2            =   zeros(nc+2)

q = plot(xc, T, xlabel="x [m]", ylabel="T [°C]", 
        title="Anfangstemperatur Verteilung", 
        markershape=:circle,label="",
        xlim=(xmin,xmax), ylim=(Tmin-10, Tmax+10))
if save_fig==0
    display(q)
end

if FD.Method.Adv==:tracers
    # Gesamtanzahl der Tracer
    nm          =   (nc)*nmx
    # Abstand der Tracer
    Δmx         =   (abs(xmin)+abs(xmax))/(nm+1)
    # x-Koordinaten der Tracer    
    xm          =   collect(xmin+Δmx:Δmx:xmax-Δmx) .+ rand(nm).*0.5*Δmx
    # Temperatur auf den Tracern
    Tm          =   zeros(nm)    
end

Nun können wir die Gleichungen zur Lösung der Advektionsgleichung programieren. 

In [163]:
# Lösen der Advektionsgleichung --------------------------------------- #
for i = 2:nt
    display(string("Time step: ",i))
    TWE[2:end-1]    .=  T
    TWE[1]          =   T[end]
    TWE[end]        =   T[1]
    
    if FD.Method.Adv==:FTCS        
        T       .= 
            TWE[2:end-1] .- (vx*Δt/2.0/Δx).*(TWE[3:end].- TWE[1:end-2])
    elseif FD.Method.Adv==:upwind
        if vx > 0
            T       .= 
                TWE[2:end-1] .- vx*Δt/Δx.*( TWE[2:end-1] .- TWE[1:end-2] )
        elseif vx < 0
            T .= 
                TWE[2:end-1] .- vx*Δt/Δx.*( TWE[3:end] .- TWE[2:end-1] ) 
        end
    elseif FD.Method.Adv==:downwind
        T   .= 
            TWE[2:end-1] .- vx*Δt/Δx.*( TWE[3:end] .- TWE[2:end-1] )
    elseif FD.Method.Adv==:lax
        T   .= ( TWE[3:end] .+ TWE[1:end-2])./2 .-
            (vx*Δt/2/Δx) .* (TWE[3:end] .- TWE[1:end-2])
    elseif FD.Method.Adv==:slf
        if i==2
            T   .= 
                TWE[2:end-1] .- vx*Δt/Δx*( TWE[3:end] .- TWE[1:end-2])
        else
            T   .= 
                TWE2[2:end-1] .- vx*Δt/Δx .* (TWE[3:end] .- TWE[1:end-2])
        end
        TWE2 .= TWE
    elseif FD.Method.Adv==:semilag
        X       .=  xc .- Δt*vx
        #spl     =   Spline1D(xcwe,TWE;k=3)
        #T       =   spl.(X)
        Itp1D_Centers2Markers!(T,X,TWE,xcwe,Δx,xmin-Δx)        
    elseif FD.Method.Adv==:tracers 
        spl     =   Spline1D(xcwe,TWE;k=1) 
        Tm      =   spl.(xm)        
        #Itp1D_Centers2Markers!(Tm,xm,TWE,xcwe,Δx,xmin-Δx)
        RK4O1D!( xm, Δt, vx, xmin, xmax )
        XMT     =   hcat(xm,Tm)        
        XMT     =   sortslices(XMT,dims=1)
        xm      .=  XMT[:,1]
        Tm      .=  XMT[:,2]
        spl2    =   Spline1D(xm,Tm;k=1)
        T       =   spl2.(xc)
        #Itp1D_Markers2Centers!( T, xc, Tm, xm, Δx, xmin)
    end
    
    display(string("ΔT=",((Tmax-maximum(T))/Tmax)*100))

    # Darstellung des Profils
    q = plot(xc, T, xlabel="x [m]", ylabel="T [°C]", 
            title="Anfangstemperatur Verteilung", 
            markershape=:circle,label="",
            xlim=(xmin,xmax), ylim=(Tmin-10, Tmax+10))
    #plot!(q,xc, T2,linstyle=:dash)
    if FD.Method.Adv==:tracers 
        plot!(xm,Tm,markershape=:circle,label="",linealpha=:0)        
    end    
    if save_fig == 1
        Plots.frame(anim)
    else
        display(q)
    end
end

# Speicher Animation ---------------------------------------------------- #
if save_fig == 1
    # Write the frames to a GIF file
    Plots.gif(anim, string( path, filename, ".gif" ), fps = 15)
    foreach(rm, filter(startswith(string(path,"00")), readdir(path,join=true)))
end
# ----------------------------------------------------------------------- #


"Time step: 2"

"ΔT=0.0"

"Time step: 3"

"ΔT=0.35546345795751577"

"Time step: 4"

...

"Time step: 124"

"ΔT=19.964711883563073"

"Time step: 125"

"ΔT=20.00168934611775"

┌ Info: Saved animation to /home/lufuchs/progs/src/julia/scripts/GeoModBox_p.jl/exercises/Correction/Results/06_1D_advection_gaussian_semilag.gif
└ @ Plots /home/lufuchs/.julia/packages/Plots/Ec1L1/src/animation.jl:156
